In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [2]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

from WordVector import WordVector

In [3]:
import data_helper
_,_,relation = data_helper.get_triplet()
relation[:5]

['cause', 'cause', 'cause', 'cause', 'cause']

In [4]:
relation = list(set(relation))
relation[:5]

['use', 'pathogen', 'capacity', 'model', 'effective']

In [5]:
wv = WordVector(relation)

Loading a pre-trained model...
Load success!


In [6]:
wv.word[:5]

['use', 'pathogen', 'capacity', 'model', 'effective']

In [7]:
wv.vector[0]

array([ 0.56000578,  1.29175377, -3.11616039,  5.55713272,  2.463449  ,
       -2.62066865,  0.98811316,  1.44295514,  1.42317593,  0.0818524 ,
        1.10450172, -2.15608644, -0.72376937, -0.82195336,  2.13655448,
        0.92574567,  2.22146559,  0.27623349,  1.50089836,  2.51285648,
        1.1609503 ,  3.31114197,  0.38355905, -0.39496148,  1.14327848,
        1.39610481,  3.7952292 ,  2.33295536,  1.99053347,  2.00406003,
       -1.33346498, -1.20852244, -2.20427084,  5.03641987, -0.88567579,
        0.08172032,  1.76758218, -0.5315519 , -1.07047224,  3.41662407,
        2.20057607, -2.83539677,  0.16929518, -4.66412735, -0.08891623,
       -1.90765929,  0.32990944, -1.12298715, -0.54387277, -1.02410674,
        2.99077439,  0.08144826,  0.47380105, -2.57034826, -2.43224144,
       -2.52519035,  6.80622435,  0.57964975, -1.51540613,  0.09802679,
        0.34399098, -0.90979362, -2.13480592, -2.13702583,  0.26595348,
        1.7715323 , -0.91994745, -0.41071165, -0.49783492,  2.92

# t-SNE: 100dim -> 3dim

In [8]:
from DimensionReduction import DimensionReduction
dr = DimensionReduction()
vec_3d = dr.TSNE(wv.vector, dim=3)
vec_3d[:3]

[t-SNE] Computing 72 nearest neighbors...
[t-SNE] Indexed 73 samples in 0.008s...
[t-SNE] Computed neighbors for 73 samples in 0.005s...
[t-SNE] Computed conditional probabilities for sample 73 / 73
[t-SNE] Mean sigma: 6.408542
[t-SNE] KL divergence after 250 iterations with early exaggeration: 112.661453
[t-SNE] Error after 1000 iterations: 2.624798


array([[ 170.52023315,  104.8395462 ,   23.09009552],
       [-171.39762878, -173.59378052,  -93.86156464],
       [-200.58738708,   59.7399826 , -164.27119446]], dtype=float32)

# Silhouette Coefficient Analysis

In [9]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
colors=["rgb(255,0,0)", "rgb(0,255,0)", "rgb(0,0,255)", "rgb(255,255,0)", "rgb(255,0,255)", 
        "rgb(0,255,255)", "rgb(127,0,255)", "rgb(255,128,0)","rgb(255,0,127)", "rgb(128,128,128)"]

In [10]:
figures = []

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig = tools.make_subplots(rows=1, cols=2,
                              print_grid=False,
                              specs=[[{'is_3d': False}, {'is_3d': True}]])

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    fig['layout']['xaxis1'].update(title='The silhouette coefficient values', range=[-0.1, 1])
   
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    fig['layout']['yaxis1'].update(title='Cluster label',
                                   showticklabels=False,
                                   range=[0, len(vec_3d) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(vec_3d)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(vec_3d, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(vec_3d, cluster_labels)
    y_lower = 10
    
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        # colors = cm.spectral(cluster_labels.astype(float) / n_clusters)

        filled_area = go.Scatter(y=np.arange(y_lower, y_upper),
                                 x=ith_cluster_silhouette_values,
                                 mode='lines',
                                 showlegend=False,
                                 line=dict(width=0.5,
                                          color=colors[i]),
                                 fill='tozerox')
        fig.append_trace(filled_area, 1, 1)
        
        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
        

    # The vertical line for average silhouette score of all the values
    axis_line = go.Scatter(x=[silhouette_avg],
                           y=[0, len(vec_3d) + (n_clusters + 1) * 10],
                           showlegend=False,
                           mode='lines',
                           line=dict(color="red", dash='dash',
                                     width =1) )

    fig.append_trace(axis_line, 1, 1)
    
    
    
    
    
    # 2nd Plot showing the actual clusters formed
    
    labeled_vec = list(zip(vec_3d, cluster_labels))
    # Labeling the clusters
    centers_ = clusterer.cluster_centers_
    for i in range(n_clusters):
        ith_cluster_vector = np.array([v for (v,c) in labeled_vec if c == i])
        # colors = matplotlib.colors.colorConverter.to_rgb(cm.spectral(float(i) / n_clusters))
        # colors = 'rgb'+str(colors)
        
        clusters = go.Scatter3d(x=ith_cluster_vector[:, 0], 
                                y=ith_cluster_vector[:, 1], 
                                z=ith_cluster_vector[:, 2],
                                showlegend=False,
                                mode='markers',
                                marker=dict(
                                    color=colors[i],
                                    size=3,
                                    line=dict(
                                        width=0.5
                                    )
                                )
                               )
        fig.append_trace(clusters, 1, 2)
        
        # Draw white circles at cluster centers
        centers = go.Scatter3d(x=centers_[i][0], 
                               y=centers_[i][1],
                               z=centers_[i][2],
                               showlegend=False,
                               mode='markers',
                               marker=dict(
                                   color=colors[i], size=5,
                                   line=dict(color='black', width=1))
                               )

        fig.append_trace(centers, 1, 2)
    
                      
        
        
        

    fig['layout'].update(title="Silhouette analysis for KMeans clustering on sample data "
                         "with n_clusters = %d" % n_clusters)
    
    figures.append(fig)

For n_clusters = 2 The average silhouette_score is : 0.802794
For n_clusters = 3 The average silhouette_score is : 0.79318
For n_clusters = 4 The average silhouette_score is : 0.678565
For n_clusters = 5 The average silhouette_score is : 0.214122
For n_clusters = 6 The average silhouette_score is : 0.233033
For n_clusters = 7 The average silhouette_score is : 0.221285
For n_clusters = 8 The average silhouette_score is : 0.218162
For n_clusters = 9 The average silhouette_score is : 0.214152
For n_clusters = 10 The average silhouette_score is : 0.21334


In [11]:
py.iplot(figures[0])

In [12]:
py.iplot(figures[1])

In [13]:
py.iplot(figures[2])